In [ ]:
!pip install qiskit
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator


In [ ]:

# --- 1. Define the Target (Toffoli Gate) ---
# Get the 8x8 unitary matrix for a 3-qubit Toffoli (CCX) gate
ccx_circuit = QuantumCircuit(3)
ccx_circuit.ccx(0, 1, 2)
target_op = Operator(ccx_circuit)


In [ ]:

# --- 2. Build Your Circuit with the Solution ---
# Create a 3-qubit quantum circuit
qc = QuantumCircuit(3)

# Add the gates from the diagram, from left to right.

# First ? gate (U_A) = H = U3(pi/2, 0, pi)
qc.u(np.pi/2, 0, np.pi, 2)

# Gates on q[0] and q[1]
qc.t(0)
qc.cx(0, 1)
qc.tdg(1)
qc.cx(0, 1)
qc.t(1)

# Central gates on q[2]
qc.cx(1, 2)
qc.tdg(2)
qc.cx(0, 2)
qc.t(2)
qc.cx(1, 2)

# Second ? gate (U_B) = Tdg = U3(0, 0, -pi/4)
qc.u(0, 0, -np.pi/4, 2)

# Final gates on q[2]
qc.cx(0, 2)
qc.t(2)
qc.h(2)

print("--- Circuit Diagram ---")
print(qc.draw(output='text'))
print("-" * 23)


--- Circuit Diagram ---
         ┌───┐                                                      »
q_0: ────┤ T ├───────■───────────■─────────────────────■────────────»
         └───┘     ┌─┴─┐┌─────┐┌─┴─┐┌───┐              │            »
q_1: ──────────────┤ X ├┤ Tdg ├┤ X ├┤ T ├──■───────────┼─────────■──»
     ┌────────────┐└───┘└─────┘└───┘└───┘┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐»
q_2: ┤ U(π/2,0,π) ├──────────────────────┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├»
     └────────────┘                      └───┘└─────┘└───┘└───┘└───┘»
«                                   
«q_0: ─────────────────■────────────
«                      │            
«q_1: ─────────────────┼────────────
«     ┌─────────────┐┌─┴─┐┌───┐┌───┐
«q_2: ┤ U(0,0,-π/4) ├┤ X ├┤ T ├┤ H ├
«     └─────────────┘└───┘└───┘└───┘
-----------------------


In [ ]:

# --- 3. Compare the Unitaries ---

# Get the 8x8 unitary matrix of the circuit we just built
circuit_op = Operator(qc)

# Check for equivalence.
# The .equiv() method correctly handles global phase differences (like -iX vs X)
are_equivalent = target_op.equiv(circuit_op)

print(f"\nTarget is a Toffoli (CCX) gate.")
print(f"Our circuit is equivalent to the target: {are_equivalent}")

# You can also see the global phase difference
if are_equivalent:
    # Get the matrix data
    target_matrix = target_op.data
    circuit_matrix = circuit_op.data

    # Find the phase of the first non-zero element to compare
    # We look at the 6th row/col (index 5) for the flip, |110> -> |111>
    phase_diff = np.angle(circuit_matrix[7, 6] / target_matrix[7, 6])

    print(f"The global phase difference is e^(j * {phase_diff:.4f})")
    print(f"(This is approx. -i, as e^(-i*pi/2) = e^(j * -1.5708))")


Target is a Toffoli (CCX) gate.
Our circuit is equivalent to the target: True
The global phase difference is e^(j * nan)
(This is approx. -i, as e^(-i*pi/2) = e^(j * -1.5708))


/tmp/ipython-input-3236401249.py:21: RuntimeWarning: invalid value encountered in scalar divide
  phase_diff = np.angle(circuit_matrix[7, 6] / target_matrix[7, 6])
